In [1]:
import numpy as np
from dolfin import *
from unray import Data, DataDisplay, render

In [2]:
mesh = Mesh("data/heart_mesh.xml.gz")
V = FunctionSpace(mesh, "P", 1)
f = interpolate(Expression("x[0]", degree=1), V)

In [3]:
fig = render(mesh.coordinates(), mesh.cells(), f.compute_vertex_values(),
             width=512, height=512)
fig.animate = False
fig

A Jupyter Widget

In [ ]:
mesh

In [ ]:
cells_array = np.asarray([0, 1, 2, 3], dtype="uint32")
coordinates_array = np.asarray([
    [0,0,0], [1,0,0], [0,1,0], [0,0,1]], dtype="float32")
density_array = np.asarray([0.1, 1.0, 0.5, 0.3], dtype="float32")

f = render(coordinates_array, cells_array, density_array,
           width=512, height=512)
f.animate = False
f

In [ ]:
f.animate = True

In [ ]:
f.animate = False

In [ ]:
cells = Data(name="cells", array=cells_array)
coordinates = Data(name="coordinates", array=coordinates_array)
density = Data(name="density", array=density_array)

In [ ]:
density_display = DataDisplay(data=density)
#density_display

In [ ]:
density_display

In [ ]:
density_display2 = DataDisplay(data=density)
density_display2

In [ ]:
density_array[:] *= 0.8
density.send_state("array")

In [ ]:
density_array

In [ ]:
density_display.send_state??

In [ ]:
density_display.name = "foo"

In [ ]:
density_display.data = cells

In [ ]:
density_display.data = density

In [ ]:
f

In [ ]:
render??

In [ ]:
rho = f.data['density']

In [ ]:
rho.array = 2 * rho.array

In [ ]:
rho